In [1]:
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [41]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    #init.run() # initialize all variables
    sess.run(init)
    result = f.eval()
    print(result)

42


In [9]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


In [10]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [11]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [12]:
x2.graph is tf.get_default_graph()

False

In [13]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [15]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [4]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [5]:
housing.data.shape

(20640, 8)

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [32]:
scaled_housing_data_plus_bias[0:2]

array([[ 1.        ,  2.34476576,  0.98214266,  0.62855945, -0.15375759,
        -0.9744286 , -0.04959654,  1.05254828, -1.32783522],
       [ 1.        ,  2.33223796, -0.60701891,  0.32704136, -0.26333577,
         0.86143887, -0.09251223,  1.04318455, -1.32284391]])

In [33]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1., 1.), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Original way
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
#training_op = tf.assign(theta, theta - learning_rate * gradients)

# Use autodiff to compute gradients automatically
#gradients = tf.gradients(mse, [theta])[0]
#training_op = tf.assign(theta, theta - learning_rate * gradients)

# Use an Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch = ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)  #the same as training_op.eval()
        
    best_theta = theta.eval()
    print(best_theta)

Epoch =  0 MSE =  10.728
Epoch =  100 MSE =  0.864979
Epoch =  200 MSE =  0.66473


Epoch =  300 MSE =  0.627966
Epoch =  400 MSE =  0.602999
Epoch =  500 MSE =  0.584399
Epoch =  600 MSE =  0.570432


Epoch =  700 MSE =  0.559894
Epoch =  800 MSE =  0.551907
Epoch =  900 MSE =  0.545824


[[ 2.06855249]
 [ 0.89367944]
 [ 0.16325891]
 [-0.32577732]
 [ 0.32878774]
 [ 0.01085616]
 [-0.04445703]
 [-0.5152812 ]
 [-0.49045202]]


In [34]:
y_train = housing.target.reshape(-1, 1)

In [38]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch*batch_size + batch_index)
    random_index = np.random.choice(m, batch_size, replace=False)  # or use indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[random_index]
    y_batch = y_train[random_index]
    return X_batch, y_batch

In [39]:
n_epochs = 1000
learning_rate = 0.005
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

theta = tf.Variable(tf.random_uniform([n+1, 1], -1., 1.), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

#gradients = 2/m * tf.matmul(tf.transpose(X), error)
#training_op = tf.assign(theta, theta - learning_rate * gradients)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()


with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        #print("Epoch = ", epoch)
        if epoch % 100 == 0:
            print("Epoch = ", epoch, "MSE = ", mse.eval(feed_dict={X: scaled_housing_data_plus_bias, y: y_train}))
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            #if epoch % 100 == 0 and batch_index % 10 == 0 :
            #    print("Epoch = ", epoch, "batch_index = ", batch_index, "MSE = ", mse.eval(feed_dict={X: X_batch, y: y_batch}))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        
    best_theta = theta.eval()
    print(best_theta)



Epoch =  0 batch_index =  70 MSE =  11.5307


Epoch =  100 batch_index =  206 MSE =  0.526935


Epoch =  200 batch_index =  206 MSE =  0.527735


Epoch =  300 batch_index =  206 MSE =  0.526361


Epoch =  400 batch_index =  206 MSE =  0.527272


KeyboardInterrupt: 

In [40]:
n_epochs = 1000
learning_rate = 0.005
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch = ", epoch, "MSE = ", mse.eval(feed_dict={X: scaled_housing_data_plus_bias, y: y_train}))
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

Epoch =  0 MSE =  4.37194


Epoch =  100 MSE =  0.533823


Epoch =  200 MSE =  0.525401


Epoch =  300 MSE =  0.529582


Epoch =  400 MSE =  0.524672


Epoch =  500 MSE =  0.52812


Epoch =  600 MSE =  0.524793


KeyboardInterrupt: 